In [1]:
%load_ext autoreload
%autoreload 1

import glob
import numpy as np
import pandas as pd
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib widget

processed_directory = './data/processed_mats/*.mat'
# processed_directory = '/Volumes/AnxietyBU/callbacks/processed/*.mat'

acceptable_call_labels = ['Call', 'Stimulus']  # any stimulus_trials containing call types NOT in this list are excluded (this includes unlabeled, which are stored as 'USV'!!)

files = [f for f in glob.glob(processed_directory)]

# or only specific files. Note: may mess up histograms, which may require data from >1 file
# files = [
#     './data/processed_mats/or60rd49-d1-20240425115050-Block1-PROCESSED.mat',
#     './data/processed_mats/or60rd49-d2-20240426114923-Block1-PROCESSED.mat'
# ]
files


['./data/processed_mats/pk81rd39-d2-20240403122742-Block6-PROCESSED.mat',
 './data/processed_mats/or54rd45-d1-20240423130940-Block0 2024-07-16  7_34 AM-PROCESSED.mat',
 './data/processed_mats/gr18bu69-d2-20240419122717-Block3-PROCESSED.mat',
 './data/processed_mats/pk63rd15-d1-202431212348-Block7 2024-07-16  9_04 AM-PROCESSED.mat',
 './data/processed_mats/gr18bu69-d2-20240419123353-Block8-PROCESSED.mat',
 './data/processed_mats/gr18bu69-d2-20240419122320-Block0-PROCESSED.mat',
 './data/processed_mats/or14pu27-d1-2024329115821-Block1-PROCESSED.mat',
 './data/processed_mats/or14pu27-d2-2024330115030-Block0-PROCESSED.mat',
 './data/processed_mats/or60rd49-d1-20240425114931-Block0 2024-07-17 10_20 AM-PROCESSED.mat',
 './data/processed_mats/pk81rd39-d2-20240403122107-Block1-PROCESSED.mat',
 './data/processed_mats/or14pu27-d2-2024330115825-Block6-PROCESSED.mat',
 './data/processed_mats/or91rd13-d1-20240711115919-Stim0-Block2 2024-07-15  8_31 AM-PROCESSED.mat',
 './data/processed_mats/gr3bu36

In [4]:
%aimport utils.deepsqueak
from utils.deepsqueak import call_mat_stim_trial_loader, multi_index_from_dict

df = pd.DataFrame()

call_types_all = pd.DataFrame()
rejected_trials_all = pd.DataFrame()

for file in files:
    calls_df, stim_trials, rejected_trials, file_info, call_types = call_mat_stim_trial_loader(file, acceptable_call_labels=['Call', 'Stimulus'], verbose=False)

    # TODO: make this a nicely editable parameter
    multi_index_info = OrderedDict()
    multi_index_info['birdname'] = file_info['birdname']
    multi_index_info['day'] = int(file_info['d'])
    multi_index_info['block'] = int(file_info['block'])

    # create multiindex: birdname, stim_trial_index, call_index
    stim_trials = multi_index_from_dict(
        stim_trials, 
        multi_index_info, 
        keep_current_index=True,
    )
    df = pd.concat((df, stim_trials), axis='rows')
    
    rejected_trials = multi_index_from_dict(
        rejected_trials, 
        multi_index_info, 
        keep_current_index=True
    )
    rejected_trials_all = pd.concat((rejected_trials_all, rejected_trials), axis='rows')

    call_types = multi_index_from_dict(
        call_types, 
        multi_index_info, 
        keep_current_index=True
    )
    call_types_all = pd.concat((call_types_all, call_types), axis='rows')

print('Rejected trials:')
rejected_trials_all

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
print(
    "Call types in rejected trials."
    + "\nLabel `USV` means an accepted call was not given a label."
    + "\nGo back to DeepSqueak & fix ths."
)

rejected_trial_call_types = call_types_all.loc[rejected_trials_all.index]
rejected_trial_call_types
# TODO: add stim index to rej trial type df (is this the first stim?)

# # see only blocks with a specific call type
#
# label = 'USV'
# label = 'Noise'
# call_types_all.loc[~np.isnan(call_types_all.loc[:, label])]

In [ ]:
df.sort_index(inplace=True)
df

In [ ]:
all_birds = list(set(df.index.get_level_values(0)))
all_birds

In [ ]:
# eliminate all block 0s - account for first loom bug

# raise Exception('Make sure you want to do this! You will need to reload the data afterward if you want block 0 back.')

blocks = df.index.get_level_values(2)
df = df[blocks != 0]

df

In [ ]:
### Plotting Parameters
day_colors = {1: "#a2cffe", 2: "#840000"}
day_labels = {1: "baseline", 2: "loom"}

stim_kwargs = dict(alpha=0.5)
call_kwargs = dict(color="black", alpha=0.5)

dpi_setting = 600  # None for jupyter output; 600+ for high-quality sharing

### Construct block-by-block response rates

In [ ]:
df_block_response_rate = df.groupby(level=["birdname", "day", "block"]).agg(
    pct_trials_responded=("n_calls", lambda x: np.count_nonzero(x) / len(x)),
    n_trials=("n_calls", lambda x: len(x)),
)
df_block_response_rate

### Plot response rate by block

In [ ]:
%%capture  
# %%capture prevents plot output

save_folder =  None
# save_folder = './data/figures/response_rates'

days = [1,2]

for birdname in all_birds:
    fig, ax = plt.subplots(dpi=dpi_setting)

    bird_data = df_block_response_rate.loc[birdname]

    for day in days:
        day_data = bird_data.loc[day]
        dc = day_colors[day]
        dl = day_labels[day]

        ax.plot(
            day_data.index,
            day_data["pct_trials_responded"],
            marker="o",
            color=dc,
            label=dl,
        )

    ax.set(
        xlabel="Block",
        ylabel="Response rate\n(% blocks with ≥ 1 response)",
        title=birdname,
        ylim=[-0.05, 1.05],
    )
    
    if save_folder is not None:
        fig.savefig(f'{save_folder}/{birdname}-response_rate.png')

### Plot Response Rates Across Days

In [ ]:
# load tempo data

tempo_data = pd.read_csv(
    "/Users/cirorandazzo/code/callback-analysis/data/loom_tempos.csv", index_col="bird"
)

tempo_colors = {"fast": "#F4CE14", "slow": "#45474B"}

tempo_data["color"] = tempo_data["family_category"].map(tempo_colors)

tempo_data

In [ ]:
df_day = df.groupby(level=["birdname", "day"]).agg(
    mean_n_calls=("n_calls", "mean"),
    mean_n_calls_excl_zero=("n_calls", lambda x: np.sum(x) / np.count_nonzero(x)),
    median_latency_s=("latency_s", lambda x: np.nanmedian(x)),  # median ignoring nans
    pct_trials_responded=("n_calls", lambda x: np.count_nonzero(x) / len(x)),
    n_trials_no_response=("n_calls", lambda x: len(x) - np.count_nonzero(x)),
    n_trials=("n_calls", lambda x: len(x)),
)
df_day

### Pre/post summary plots

- n_calls
- latency
- response rate

In [ ]:
%%capture

import matplotlib.patches as mpatches

%aimport utils.plot
from utils.plot import plot_pre_post

savefig = True
highlight_bird = None
# highlight_bird = 'pk81rd39'

to_plot_pre_post = {
    "n_calls" : dict(
        fieldname="mean_n_calls",
        ax_kwargs = dict(
            title="Mean Calls/Trial Across Days",
            ylabel="Mean # Calls per Trial",
            ylim=[-0.1,3]
        )
    ),
    
    "n_calls_excl_zero" : dict(
        fieldname="mean_n_calls_excl_zero",
        ax_kwargs = dict(
            title="Mean Calls/Trial Across Days (Excl. 0)",
            ylabel="Mean # Calls per Trial",
            ylim=[0.9,3]
        )
    ),
    
    "latency" : dict(
        fieldname="median_latency_s", 
        ax_kwargs = dict(
            title="Median Latency Across Days",
            ylabel="Median Latency (s)",
        )
    ),
    
    "response_rate" : dict(
        fieldname="pct_trials_responded", 
        ax_kwargs = dict(
            yticks=np.arange(0, 1.2, 0.2),
            title="Response Rate Across Days",
            ylabel="% Trials with ≥1 Response",
        )
    ),
}

# proxy artists for legend
handles = [
    mpatches.Patch(color=color, label=speed)
    for speed, color in tempo_colors.items()
]

for fname, parameters in to_plot_pre_post.items():

    fig, ax = plt.subplots(figsize=[4, 6], dpi=dpi_setting)

    ax_kwargs = parameters.get('ax_kwargs', {})

    ax = plot_pre_post(
        df_day, 
        fieldname=parameters['fieldname'], 
        ax=ax,
        color=tempo_data,
        plot_kwargs={'marker': 'o'},
        add_bird_label=True,  # default: false. adds bird ids on plot
    )

    if highlight_bird is not None:
        fname += f"-{highlight_bird}"

        ax = plot_pre_post(
            df_day.xs(highlight_bird, level="birdname", drop_level=False),
            fieldname=parameters['fieldname'], 
            ax=ax,
            color='green',
            plot_kwargs={'marker': 'o', 'linestyle':'--'}
        )

    ax.set(
        xticks=[1, 2],
        # xlim=[0.75, 2.25],  # if no bird labels
        xlim=[0.75, 2.5],  # if bird labels
        xticklabels=["baseline", "loom"],
        # yticks=np.arange(0, 1.2, 0.2),
        **ax_kwargs,
    )

    if savefig:
        fig.savefig(f"./data/figures/summary/{fname}.png", bbox_inches="tight")

        # ax.legend(handles=handles)

        # fig.savefig(f"./data/figures/summary/{fname}-legend.png", bbox_inches="tight")

### Deltas & Correlation

In [ ]:
df_day

In [ ]:
# takes d2 - d1
d2minusD1 = lambda x: x.loc[:, 2] - x.loc[:, 1]

delta_df = df_day.groupby("birdname").agg(
    d_pct_trials_responded=("pct_trials_responded", d2minusD1),
    d_mean_n_calls=("mean_n_calls", d2minusD1),
    d_median_latency_s=("median_latency_s", d2minusD1),
)

# get song tempo & callback latency
delta_df["song_tempo"] = delta_df.index.map(tempo_data["median song tempo"])
# delta_df["callback_latency"] = delta_df.index.map(tempo_data["callback latency"])

delta_df

In [ ]:
axis_labels_by_field = {
    "song_tempo": "Median song tempo\n(syl/s)",
    "d_pct_trials_responded": "$\Delta$ % trials with ≥1 response",
    "d_mean_n_calls": "$\Delta$ mean call count/trial",
    "d_median_latency_s": "$\Delta$ median latency (s)",
}


correlations_to_plot = {
    "C--song_tempo-response_rate": dict(
        x_field="song_tempo",
        y_field="d_pct_trials_responded",
        ax_kwargs=dict(
            title="$\Delta$ response rate vs. song tempo",
        ),
    ),
    "C--song_tempo-n_calls": dict(
        x_field="song_tempo",
        y_field="d_mean_n_calls",
        ax_kwargs=dict(
            title="$\Delta$ mean calls/trial vs. Song tempo",
        ),
    ),
    "C--song_tempo-latency": dict(
        x_field="song_tempo",
        y_field="d_median_latency_s",
        ax_kwargs=dict(
            title="$\Delta$ median latency vs. Song tempo",
        ),
    ),
}

corr_kwargs = dict(
    color="k",
    marker="o",
)

In [ ]:
%%capture

savefig = True

for fname, parameters in correlations_to_plot.items():
    x_field = parameters["x_field"]
    y_field = parameters["y_field"]

    fig, ax = plt.subplots(dpi=dpi_setting)
    ax.scatter(delta_df[x_field], delta_df[y_field], **corr_kwargs)

    ax.set(
        xlabel=axis_labels_by_field[x_field],
        ylabel=axis_labels_by_field[y_field],
        **parameters["ax_kwargs"],
    )

    if savefig:
        fig.savefig(f"./data/figures/correlation/{fname}.png", bbox_inches="tight")

In [ ]:
savefig = False

fig, ax = plt.subplots(dpi=dpi_setting)

ax.scatter(delta_df["song_tempo"], delta_df[""], **corr_kwargs)

ax.set(
    # xlim=[4.5, 8],
)

if savefig:
    fig.savefig(f"./data/figures/correlation/{fname}.png", bbox_inches="tight")

In [ ]:
plt.close("all")

## Plot Rasters

### Raster by block

In [ ]:
%%capture  
# %%capture prevents plot output

%aimport utils.plot
from utils.plot import plot_callback_raster

save_folder =  None
save_folder = './data/figures/callback_rasters_by_block'

# every bird/day/block
unique_conditions = list(set([a[0:3] for a in df.index]))

## or select a subset
# unique_conditions = [
#     ('or14pu27', 1, 1),
#     ('or14pu27', 2, 1),
#     ('or54rd45', 1, 1),
#     ('or54rd45', 2, 1),
# ]

# figs = {}

for bird, day, block in unique_conditions:

    fig = plt.figure()
    ax = fig.subplots()

    data = df.loc[(bird, day, block)]
    
    title_str = f'{bird}-d{day}-b{block}'

    stim_kwargs['color'] = day_colors[day]

    plot_callback_raster(
        data,
        ax=ax,
        title = title_str,
        plot_stim_blocks = False,
        show_legend = True,
        call_kwargs = call_kwargs,
        stim_kwargs = stim_kwargs,
    )

    ax.set_xlim([-0.1, 3])

    # figs[title_str] = fig

    if save_folder is not None:
        fig.savefig(f'{save_folder}/{title_str}.png')

### Raster by day

In [ ]:
%%capture  
# %%capture prevents plot output

%aimport utils.plot
from utils.plot import plot_callback_raster_multiblock

save_folder =  None
save_folder = './data/figures/callback_rasters_multiblock'

# every bird/day
unique_conditions = list(set([a[0:2] for a in df.index]))

for bird, day in unique_conditions:
    data = df.loc[(bird, day)]

    title_str = f"{bird}-d{day}"

    stim_kwargs = dict(color=day_colors[day], alpha=0.5, edgecolor=None)
    call_kwargs = dict(color="black", alpha=0.5, edgecolor=None)

    fig = plt.figure()
    ax = fig.subplots()

    plot_callback_raster_multiblock(
        data,
        ax=ax,
        plot_hlines=True,
        show_block_axis=True,
        show_legend=False,
        xlim=[-0.1, 3],
        stim_kwargs = stim_kwargs,
        call_kwargs = call_kwargs,
        title = title_str,
    )

    if save_folder is not None:
        fig.savefig(f'{save_folder}/{title_str}.png')

## Violin plots

In [ ]:
# NOTE: violin plot only works for days = [1,2] right now

days = [1, 2]
width = 0.75

In [ ]:
%%capture  
# %%capture prevents plot output

%aimport utils.plot
from utils.plot import plot_violins_by_block

save_folder =  None
save_folder = './data/figures/n_calls'
# save_folder = './data/figures/n_calls-norm'

for bird in all_birds:
    fig, ax = plt.subplots(dpi=dpi_setting)
    title_str = bird

    ax = plot_violins_by_block(
            df.loc[bird],
            field="n_calls",
            ax=ax,
            days=days,
            day_colors=day_colors,
            width=width,
            dropna=False,
    )

    ax.set(
        xlim=[-0.5,9.5],
        xticks= np.arange(0,10),
        # ylim=[-.5, 8],
        xlabel='Block',
        ylabel='Calls per stimulus',
        title=title_str,
    )
    
    if save_folder is not None:
        fig.savefig(f'{save_folder}/{title_str}.png')

In [ ]:
%%capture  
# %%capture prevents plot output

%aimport utils.plot
from utils.plot import plot_violins_by_block

save_folder =  None
save_folder = './data/figures/latency'
# save_folder = './data/figures/latency-norm'

for bird in all_birds:
    fig, ax = plt.subplots(dpi=dpi_setting)
    title_str = bird

    ax = plot_violins_by_block(
            df.loc[bird],
            field="latency_s",
            ax=ax,
            days=days,
            day_colors=day_colors,
            width=width,
            dropna=True,
    )

    ax.set(
        xlim=[-0.5,9.5],
        xticks= np.arange(0,10),
        # ylim=[0, 2.5],
        xlabel='Block',
        ylabel='Latency to first call (s)',
        title=title_str,
    )
    
    if save_folder is not None:
        fig.savefig(f'{save_folder}/{title_str}.png')

## Histograms

All blocks merged

### Latency

In [ ]:
%%capture

# index levels: 'birdname', 'day', 'block', 'stims_index'
# idx = pd.IndexSlice
# this_bird = df.loc[idx[birdname, :, :, :]]

save_folder = None
save_folder = './data/figures/histograms/latency'

%aimport utils.plot
from utils.plot import plot_group_hist

for bird in all_birds:

    fig, ax = plt.subplots(dpi=dpi_setting)

    plot_group_hist(
        df.loc[bird],
        field="latency_s",
        grouping_level="day",
        group_colors=day_colors,
        alt_labels={1: "baseline", 2: "loom"},
        ax=ax,
        density=True,
        ignore_nan=True,
        histogram_kwargs={
            "range": (0, 1.5),
            "bins": 40,
        },
        stair_kwargs={
            1: {"hatch": "/"},
            2: {"hatch": "\\"},
        },
    )

    ax.set(
        title=f"{bird}: latency to first call",
        xlabel="Latency (s)",
    )

    if save_folder is not None:
        fig.savefig(f'{save_folder}/{bird}-latency.png')

In [ ]:
%%capture

save_folder = None
save_folder = "./data/figures/histograms/n_calls"
# save_folder = './data/figures'

%aimport utils.plot
from utils.plot import plot_group_hist

for bird in all_birds:

    fig, ax = plt.subplots(dpi=dpi_setting)

    plot_group_hist(
        df.loc[bird],
        field="n_calls",
        grouping_level="day",
        group_colors=day_colors,
        alt_labels={1: "baseline", 2: "loom"},
        ax=ax,
        density=True,
        ignore_nan=False,
        histogram_kwargs={
            "range": (-0.5, 9.5),
            "bins": 10,
        },
        stair_kwargs={
            1: {"hatch": "/"},
            2: {"hatch": "\\"},
        },
    )

    ax.set(
        title=f"{bird}: number of calls per trial",
        xlabel="# of calls",
        xticks=list(range(0, 10)),
    )

    if save_folder is not None:
        fig.savefig(f"{save_folder}/{bird}-ncalls.png")

In [ ]:
# to generate legend

import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

# day_colors = {1: "#a2cffe", 2: "#840000"}
# day_labels = {1: "baseline", 2: "loom"}
alpha = 0.5

fig, ax = plt.subplots(dpi=dpi_setting)

handles = []
for k, v in day_labels.items():
    handles.append(
        Rectangle([0, 0], 0, 0, color=day_colors[k], alpha=alpha, label=day_labels[k])
    )

ax.legend(handles=handles)

plt.show()

TODO

In [ ]:
# TODO: maybe replace some hacky indexing stuff with `groupby`. eg:
df["n_calls"].groupby(level=["birdname", "day"]).mean()